# Inference Sample for OpenFold

SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved. \
SPDX-License-Identifier: LicenseRef-NvidiaProprietary

NVIDIA CORPORATION, its affiliates and licensors retain all intellectual property and proprietary rights in and to this material, related documentation and any modifications thereto. Any use, reproduction, disclosure or distribution of this material and related documentation without an express license agreement from NVIDIA CORPORATION or its affiliates is strictly prohibited.

### Prerequisite

- Linux OS
- Pascal, Volta, Turing, or an NVIDIA Ampere architecture-based GPU.
- NVIDIA Driver
- Docker

#### Import

Components for inferencing are part of the BioNeMo OpenFold source code. This notebook demonstrates the use of these components.


In [16]:
import os
import numpy as np
import py3Dmol
from pytriton.client import ModelClient
from bionemo.triton.utils import read_bytes_from_filepaths
from typing import List

In [17]:
BIONEMO_HOME = os.environ['BIONEMO_HOME']
TEST_DATA_DIR = os.path.join(BIONEMO_HOME, 'examples/tests/test_data/openfold_data')

# Start inference server
Before starting the inference server, users should download OpenFold NeMo checkpoints through `download_model.py` with the following command.

```python download_models.py openfold_finetuning_inhouse --download_dir ${BIONEMO_HOME}/models```

Now, we can initialize the inference server.

`python examples/protein/openfold/nbs/infer_server.py`


# Inputs
## Set input sequences and optional MSAs
In case users are interested in inference with MSAs, precomputed MSAs for example sequences are available through `./examples/protein/openfold/scripts/download_sample_data.sh`.

In [18]:
sequences = [
    'GASTATVGRWMGPAEYQQMLDTGTVVQSSTGTTHVAYPADIDAFGKQAKNGAMYVEFDVPEKSLVPTNEGWAKIVGPDSIEGRLAKRKGLPVPEMPTAENITVRGEKINGEVEAKC',
    'MTDSIKTLSAHRSFGGVQHFHEHASREIGLPMRFAAYLPPQAEHGKVPALLYLAGLTCNEETFMVKAGAQRLAAELGIALIAPDTSPRGAHIDGESTSWDFGVGAGFYLDATAAPWAPNWRMESYLVDELLPLLAKTLPIDGDRIGVFGHSMGGHGALTLALRHPGLFKSLSAFAPICAPTQCPWGHKAFTGYLGADTTRWIEHDATVLMQHQPVAPYPAGILIDQGLADKFLAEQLHPHLLEDACRAIGQPLTLRRHEGYDHGYYFVQSFMADHLAHHAQILNGAIGRPAARQP',
]

### Prepare for optional MSA inputs

In [19]:
# Use sample MSA inputs
msa_a3m_filepaths: List[List[str]] = [
    ['7ZHL_A_uniref90_alignment.a3m'],  # can have non-uniform number of msa per sequence
    ['7YVT_B_mgnify_alignment.a3m', '7YVT_B_smallbfd_alignment.a3m', '7YVT_B_uniref90_alignment.a3m'],
]
msa_a3m_filepaths = [[os.path.join(TEST_DATA_DIR, 'inference/msas', msa) for msa in msas] for msas in msa_a3m_filepaths]

# Skip MSA inputs
# msa_a3m_filepaths: List[List[str]] = [[], []]

In [20]:
# Test inference inputs to ensure the same number of samples
if len(sequences) != len(msa_a3m_filepaths):
    raise ValueError(
        f'Sequences and msa_a3m_filepaths have inconsistent number of inference samples. Got {len(sequences), len(msa_a3m_filepaths)}'
    )

# Read contents from msa_a3m_filepaths and batch into array for Triton 
max_length = max(len(msa_a3m_filepaths_) for msa_a3m_filepaths_ in msa_a3m_filepaths)

if max_length:
    msa_a3m_file_contents = []
    for msa_a3m_filepaths_ in msa_a3m_filepaths:
        msa_a3m_file_contents_: List[bytes] = read_bytes_from_filepaths(*msa_a3m_filepaths_)  # reading
        msa_a3m_file_contents_ += [''] * (max_length - len(msa_a3m_file_contents_))  # batching
        msa_a3m_file_contents.append(msa_a3m_file_contents_)
else:
    msa_a3m_file_contents = [[''],] * len(msa_a3m_filepaths)

# Convert input for Triton inference
sequences = np.array(sequences)
sequences = np.char.encode(sequences, 'utf-8')
msa_a3m_file_contents = np.array(msa_a3m_file_contents)
msa_a3m_file_contents = np.char.encode(msa_a3m_file_contents, 'utf-8')

# Batching
sequences_batch = sequences[np.newaxis, ...]
msa_a3m_file_contents_batch = msa_a3m_file_contents[np.newaxis, ...]

###  Use ModelClient to run inference on the server

OpenFold prediction is written to `OUTPUT_DIR`. Template search and feature dumping is not implemented on this Jupyter Notebook example.

In [21]:
# inference through Triton server
with ModelClient("localhost", "bionemo_openfold") as client:
    output_pdb_strings = client.infer_batch(
        sequences_batch=sequences_batch,
        msa_a3m_file_contents_batch=msa_a3m_file_contents_batch,
    )
    output_pdb_strings = output_pdb_strings['output_pdb_string']
    output_pdb_strings = np.char.decode(output_pdb_strings.astype('bytes'), 'utf-8')
    output_pdb_strings = output_pdb_strings.tolist()[0]


## Visualize the prediction results

In [22]:
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(output_pdb_strings[0])
view.setStyle({'model': -1}, {"cartoon": {'color': 'purple'}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol